In [1]:
# Prepare gym for agent
import gym
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Getting enviroment
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [2]:
print(env.action_space)
env.reset()
state, reward, done, info = env.step(env.action_space.sample())
print(state)

Discrete(2)
[ 0.02881814 -0.18446311 -0.04327379  0.30948815]


Two possible actions [0,1]
State descrived by [position of cart, velocity of cart, angle of pole, rotation rate of pole].

Neural Network implementation to approximate the Q action value function.

## Experience Replay memory implementation.

In [94]:
from collections import deque
class Memory_exp_replay:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def add_timestep(self, env_reaction):
        # St, At, Rt1, St1.
        self.buffer.append(env_reaction)
        
    def sample_random(self, sample_size, debug=False):
        # Replace=False, important so it doesn't repeat a sample on the batch
        indexs = np.random.choice(a=np.arange(len(self.buffer)), size=sample_size, replace=False) 
        if debug:
            print(indexs)
        states = list()
        actions = list()
        rewards = list()
        next_states = list()
        for index in indexs:
            states.append(self.buffer[index][0])
            actions.append(self.buffer[index][1])
            rewards.append(self.buffer[index][2])
            next_states.append(self.buffer[index][3])
        batch = [np.array(states), np.array(actions), np.array(rewards), np.array(next_states)]
        return batch

Testing out the Memory Buffer.

In [95]:
# Initliaze memory
def populate_memory(env, exp_replay_memory, batch_size_Q, debug=False):
    if debug: 
        print('Order of enviroment reactions')
    content = 0
    s_t1 = env.reset()
    while content < batch_size_Q:
        s_t = s_t1 
        a_t = env.action_space.sample()
        s_t1, r_t1, done, info = env.step(a_t)
        if done:
            s_t1 = np.zeros(s_t.shape)
            if debug: 
                print(content, s_t)
            exp_replay_memory.add_timestep((s_t, a_t, r_t1, s_t1))
            s_t = env.reset()
        else:
            if debug: 
                print(content, s_t)
            exp_replay_memory.add_timestep((s_t, a_t, r_t1, s_t1))
        content += 1
    return exp_replay_memory

memory_size = 100000
batch_size_Q = 20

# Intantiate memory and populate
exp_replay = Memory_exp_replay(memory_size)
exp_replay = populate_memory(env, exp_replay, batch_size_Q, debug=True)

print('Random sample from Memory Buffer')
states, actions, rewards, next_states = exp_replay.sample_random(2, debug=True)
for state in states:
    print(state)

Order of enviroment reactions
0 [-0.0279756  -0.01825218 -0.03759038 -0.01097499]
1 [-0.02834064  0.17738814 -0.03780988 -0.31527713]
2 [-0.02479288  0.3730277  -0.04411542 -0.61964019]
3 [-0.01733233  0.56873714 -0.05650823 -0.92588446]
4 [-0.00595758  0.76457483 -0.07502592 -1.2357767 ]
5 [ 0.00933391  0.96057648 -0.09974145 -1.55098904]
6 [ 0.02854544  0.76678241 -0.13076123 -1.29101807]
7 [ 0.04388109  0.96330187 -0.15658159 -1.62161505]
8 [ 0.06314713  0.7703323  -0.18901389 -1.38154894]
9 [0. 0. 0. 0.]
10 [-0.00125512 -0.14961133 -0.00463465  0.32100127]
11 [-0.00424735 -0.34466697  0.00178538  0.61221901]
12 [-0.01114069 -0.14957002  0.01402976  0.32009895]
13 [-0.01413209  0.04534935  0.02043174  0.03187329]
14 [-0.0132251  -0.15005956  0.0210692   0.33093197]
15 [-0.0162263   0.04475624  0.02768784  0.04496702]
16 [-0.01533117 -0.15075157  0.02858718  0.34625552]
17 [-0.0183462  -0.34626825  0.03551229  0.6478142 ]
18 [-0.02527157 -0.54186649  0.04846858  0.951465  ]
19 [-0.03

### Memory working well.

## Q Value Neural Network inference.

In [130]:
# Defining NN for the action value function approximation.
class NN_Q_approx:
    def __init__(self, learning_rate, hidden_size, state_space, action_space, name):
        self.learning_rate = learning_rate
        self.state_space = state_space
        self.action_space = action_space
        self.name = name
        with tf.variable_scope(self.name):
            self.inputs = tf.placeholder(dtype=tf.float32, shape=[None, self.state_space], name='state_input')
            
            # Placeholder to choose the action value for the action done.
            self.actions = tf.placeholder(dtype=tf.int32, shape=[None], name='action_output')
            actions_oh = tf.one_hot(self.actions, self.action_space)
            
            self.target_Q = tf.placeholder(dtype=tf.float32, shape= [None], name='target_Q')
            
            self.fc1 = tf.contrib.layers.fully_connected(self.inputs, hidden_size)
            self.fc2 = tf.contrib.layers.fully_connected(self.fc1, hidden_size)
            self.output = tf.contrib.layers.fully_connected(self.fc2, self.action_space, activation_fn=None)
            # Until here for action value prediction.
            
            # Q size [m, action_space] -> sum, size [m, 1]
            self.Q = tf.reduce_sum(tf.multiply(self.output, actions_oh), axis=1)
            
            self.loss = tf.reduce_mean(tf.square(self.Q - self.target_Q))
            self.opt = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)     
            

Defining parameters for neural network and replay memory, to test out how NN performs to learn Q values.

In [220]:
# NN parameters.
learning_rate_nn = 0.0001
hidden_size = 64
state_space = len(state)
action_space = env.action_space.n

# Memory parameters.
memory_size = 100000
batch_size_Q = 32

# Training parameters.
max_episodes = 700
max_steps_episode = 300
gamma = 1

# Exploration parameters
max_epsilon = 1.0
min_epsilon = 0.01
decay_rate = 0.01  

Defining Epislon-Greedy Policy.

In [221]:
def epsilon_greedy(epsilon, q_value):
    probabilities = np.ones(q_value[0].shape)*epsilon/action_space
    probabilities[np.argmax(q_value[0])] = 1 - epsilon + (epsilon/action_space)
    action = np.random.choice(a=np.arange(action_space), size=1, p=probabilities)
    return int(action)

Code to run training.

In [222]:
# Intantiate memory and populate
exp_replay = Memory_exp_replay(memory_size)
exp_replay = populate_memory(env, exp_replay, batch_size_Q)

# Initiliaze TensorFlow Graph
tf.reset_default_graph()
Q_NN = NN_Q_approx(learning_rate=learning_rate_nn, hidden_size=hidden_size, state_space=state_space, action_space=action_space, name='Q_action_value')

training_rewards = list()
training_loss = list()
with tf.Session() as sess:
    # Initializa variables 
    sess.run(tf.global_variables_initializer())
    ep_iter = 0
    
    for i in range(1, max_episodes+1):
        # Intiliazing variables for episode.
        s_t1 = env.reset()
        time_step = 0
        episode_reward = 0
        
        while time_step < max_steps_episode:
            # Watch it learn.
            # env.render()
            
            s_t = s_t1
            # Controling epsilon.
            ep_iter += 1
            epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*i) 
        
            # Find Q Values and select action under E-greedy policy.
            q_values = sess.run(Q_NN.output, feed_dict={Q_NN.inputs:np.array(s_t).reshape((1, -1))})
            a_t = epsilon_greedy(epsilon, q_values)

            s_t1, r_t1, done, info = env.step(a_t)
    
            episode_reward += gamma*r_t1
            time_step += 1
            
            if done:
                # Terminating episode
                time_step = max_steps_episode
                s_t1 = np.zeros(state_space)
                # Update memory
                exp_replay.add_timestep((s_t, a_t, r_t1, s_t1))

                # Tracking information
                training_rewards.append((i, episode_reward))
                training_loss.append((i, loss))
                print('Episode: {}'.format(i),
                      'Total reward: {:.4f}'.format(episode_reward),
                      'Training loss: {:.4f}'.format(loss),
                      'Epsilon: {:.4f}'.format(epsilon))
            else:
                # Update memory
                exp_replay.add_timestep((s_t, a_t, r_t1, s_t1))
            
            # Train network.
            states, actions, rewards, next_states = exp_replay.sample_random(batch_size_Q)
            estiamted_Qs = sess.run(Q_NN.output, feed_dict = {Q_NN.inputs:next_states})
            found_initial = (next_states == np.zeros(states[0].shape)).all(axis=1)
            estiamted_Qs[found_initial] = np.zeros(estiamted_Qs[0].shape)
            
            target_Qs = rewards + gamma*np.max(estiamted_Qs, axis=1)
            
            feed_dict = {Q_NN.inputs:states, Q_NN.actions:actions, Q_NN.target_Q: target_Qs}
            loss, _ = sess.run([Q_NN.loss, Q_NN.opt], feed_dict)
                    

Episode: 1 Total reward: 12.0000 Training loss: 1.0972 Epsilon: 0.9901
Episode: 2 Total reward: 30.0000 Training loss: 1.1010 Epsilon: 0.9804
Episode: 3 Total reward: 24.0000 Training loss: 1.0963 Epsilon: 0.9707
Episode: 4 Total reward: 22.0000 Training loss: 1.0845 Epsilon: 0.9612
Episode: 5 Total reward: 14.0000 Training loss: 1.1283 Epsilon: 0.9517
Episode: 6 Total reward: 31.0000 Training loss: 1.2213 Epsilon: 0.9423
Episode: 7 Total reward: 16.0000 Training loss: 1.0767 Epsilon: 0.9331
Episode: 8 Total reward: 20.0000 Training loss: 1.2465 Epsilon: 0.9239
Episode: 9 Total reward: 14.0000 Training loss: 1.2262 Epsilon: 0.9148
Episode: 10 Total reward: 9.0000 Training loss: 1.2384 Epsilon: 0.9058
Episode: 11 Total reward: 24.0000 Training loss: 1.2937 Epsilon: 0.8969
Episode: 12 Total reward: 11.0000 Training loss: 1.4241 Epsilon: 0.8881
Episode: 13 Total reward: 14.0000 Training loss: 1.3945 Epsilon: 0.8793
Episode: 14 Total reward: 19.0000 Training loss: 1.3940 Epsilon: 0.8707
Ep

Episode: 118 Total reward: 10.0000 Training loss: 51.9768 Epsilon: 0.3142
Episode: 119 Total reward: 10.0000 Training loss: 22.8515 Epsilon: 0.3112
Episode: 120 Total reward: 10.0000 Training loss: 55.6325 Epsilon: 0.3082
Episode: 121 Total reward: 12.0000 Training loss: 27.8523 Epsilon: 0.3052
Episode: 122 Total reward: 9.0000 Training loss: 36.9259 Epsilon: 0.3023
Episode: 123 Total reward: 11.0000 Training loss: 23.3286 Epsilon: 0.2994
Episode: 124 Total reward: 11.0000 Training loss: 4.2243 Epsilon: 0.2965
Episode: 125 Total reward: 9.0000 Training loss: 46.6482 Epsilon: 0.2936
Episode: 126 Total reward: 12.0000 Training loss: 56.4964 Epsilon: 0.2908
Episode: 127 Total reward: 11.0000 Training loss: 42.2280 Epsilon: 0.2880
Episode: 128 Total reward: 13.0000 Training loss: 31.9865 Epsilon: 0.2853
Episode: 129 Total reward: 10.0000 Training loss: 15.3828 Epsilon: 0.2825
Episode: 130 Total reward: 10.0000 Training loss: 63.5750 Epsilon: 0.2798
Episode: 131 Total reward: 10.0000 Traini

Episode: 234 Total reward: 52.0000 Training loss: 17.9669 Epsilon: 0.1054
Episode: 235 Total reward: 21.0000 Training loss: 10.8593 Epsilon: 0.1044
Episode: 236 Total reward: 18.0000 Training loss: 5.0173 Epsilon: 0.1035
Episode: 237 Total reward: 33.0000 Training loss: 10.2487 Epsilon: 0.1025
Episode: 238 Total reward: 31.0000 Training loss: 30.8790 Epsilon: 0.1016
Episode: 239 Total reward: 22.0000 Training loss: 28.3826 Epsilon: 0.1007
Episode: 240 Total reward: 19.0000 Training loss: 25.8632 Epsilon: 0.0998
Episode: 241 Total reward: 17.0000 Training loss: 25.1769 Epsilon: 0.0989
Episode: 242 Total reward: 20.0000 Training loss: 11.5597 Epsilon: 0.0980
Episode: 243 Total reward: 22.0000 Training loss: 2.2708 Epsilon: 0.0972
Episode: 244 Total reward: 30.0000 Training loss: 11.5500 Epsilon: 0.0963
Episode: 245 Total reward: 32.0000 Training loss: 17.1455 Epsilon: 0.0954
Episode: 246 Total reward: 33.0000 Training loss: 18.3891 Epsilon: 0.0946
Episode: 247 Total reward: 16.0000 Train

Episode: 346 Total reward: 36.0000 Training loss: 41.3974 Epsilon: 0.0411
Episode: 347 Total reward: 45.0000 Training loss: 71.1542 Epsilon: 0.0408
Episode: 348 Total reward: 40.0000 Training loss: 147.0573 Epsilon: 0.0405
Episode: 349 Total reward: 39.0000 Training loss: 43.9660 Epsilon: 0.0402
Episode: 350 Total reward: 40.0000 Training loss: 152.4601 Epsilon: 0.0399
Episode: 351 Total reward: 34.0000 Training loss: 85.1219 Epsilon: 0.0396
Episode: 352 Total reward: 30.0000 Training loss: 4.6435 Epsilon: 0.0393
Episode: 353 Total reward: 46.0000 Training loss: 81.8582 Epsilon: 0.0390
Episode: 354 Total reward: 31.0000 Training loss: 53.1545 Epsilon: 0.0387
Episode: 355 Total reward: 33.0000 Training loss: 27.1820 Epsilon: 0.0384
Episode: 356 Total reward: 29.0000 Training loss: 17.5429 Epsilon: 0.0382
Episode: 357 Total reward: 21.0000 Training loss: 3.5933 Epsilon: 0.0379
Episode: 358 Total reward: 25.0000 Training loss: 28.5159 Epsilon: 0.0376
Episode: 359 Total reward: 20.0000 Tra

Episode: 457 Total reward: 53.0000 Training loss: 87.2420 Epsilon: 0.0203
Episode: 458 Total reward: 45.0000 Training loss: 82.0843 Epsilon: 0.0202
Episode: 459 Total reward: 97.0000 Training loss: 5.7768 Epsilon: 0.0201
Episode: 460 Total reward: 34.0000 Training loss: 15.2964 Epsilon: 0.0200
Episode: 461 Total reward: 37.0000 Training loss: 2.9448 Epsilon: 0.0199
Episode: 462 Total reward: 40.0000 Training loss: 109.7427 Epsilon: 0.0198
Episode: 463 Total reward: 44.0000 Training loss: 106.8622 Epsilon: 0.0197
Episode: 464 Total reward: 38.0000 Training loss: 17.4033 Epsilon: 0.0196
Episode: 465 Total reward: 66.0000 Training loss: 2.8568 Epsilon: 0.0195
Episode: 466 Total reward: 75.0000 Training loss: 126.7397 Epsilon: 0.0194
Episode: 467 Total reward: 53.0000 Training loss: 3.1447 Epsilon: 0.0193
Episode: 468 Total reward: 119.0000 Training loss: 152.7822 Epsilon: 0.0192
Episode: 469 Total reward: 67.0000 Training loss: 127.7525 Epsilon: 0.0191
Episode: 470 Total reward: 49.0000 T

Episode: 567 Total reward: 200.0000 Training loss: 83.5080 Epsilon: 0.0134
Episode: 568 Total reward: 200.0000 Training loss: 7.6693 Epsilon: 0.0134
Episode: 569 Total reward: 200.0000 Training loss: 315.0185 Epsilon: 0.0133
Episode: 570 Total reward: 200.0000 Training loss: 40.1751 Epsilon: 0.0133
Episode: 571 Total reward: 200.0000 Training loss: 14.1972 Epsilon: 0.0133
Episode: 572 Total reward: 200.0000 Training loss: 5.6741 Epsilon: 0.0132
Episode: 573 Total reward: 200.0000 Training loss: 8.8895 Epsilon: 0.0132
Episode: 574 Total reward: 200.0000 Training loss: 12.6301 Epsilon: 0.0132
Episode: 575 Total reward: 200.0000 Training loss: 110.9623 Epsilon: 0.0132
Episode: 576 Total reward: 200.0000 Training loss: 10.2360 Epsilon: 0.0131
Episode: 577 Total reward: 200.0000 Training loss: 12.0949 Epsilon: 0.0131
Episode: 578 Total reward: 200.0000 Training loss: 452.7048 Epsilon: 0.0131
Episode: 579 Total reward: 200.0000 Training loss: 256.8006 Epsilon: 0.0130
Episode: 580 Total rewar

Episode: 677 Total reward: 200.0000 Training loss: 4.2799 Epsilon: 0.0111
Episode: 678 Total reward: 200.0000 Training loss: 12.3940 Epsilon: 0.0111
Episode: 679 Total reward: 200.0000 Training loss: 8.3464 Epsilon: 0.0111
Episode: 680 Total reward: 200.0000 Training loss: 1695.3616 Epsilon: 0.0111
Episode: 681 Total reward: 200.0000 Training loss: 1116.5195 Epsilon: 0.0111
Episode: 682 Total reward: 200.0000 Training loss: 22.4020 Epsilon: 0.0111
Episode: 683 Total reward: 200.0000 Training loss: 24.1184 Epsilon: 0.0111
Episode: 684 Total reward: 200.0000 Training loss: 5.3314 Epsilon: 0.0111
Episode: 685 Total reward: 200.0000 Training loss: 205.1502 Epsilon: 0.0110
Episode: 686 Total reward: 200.0000 Training loss: 59.9110 Epsilon: 0.0110
Episode: 687 Total reward: 200.0000 Training loss: 24.5963 Epsilon: 0.0110
Episode: 688 Total reward: 200.0000 Training loss: 23.2641 Epsilon: 0.0110
Episode: 689 Total reward: 200.0000 Training loss: 38.1052 Epsilon: 0.0110
Episode: 690 Total rewa

In [223]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

eps, rews = np.array(training_rewards).T
smoothed_rews = running_mean(rews, 10)
plt.plot(eps[-len(smoothed_rews):], smoothed_rews)
plt.plot(eps, rews, color='blue', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.show()

In [224]:
eps, loss = np.array(training_loss).T
plt.plot(loss, color='blue')
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.show()